# Average 3 Fold predictions

In [7]:
import json
import numpy as np
from pathlib import Path
import pandas as pd

fold_paths = [
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/2023-09-09T233134_CellViT-SAM-H-Fold-1",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/2023-09-10T014745_CellViT-SAM-H-Fold-2",
    "/homes/fhoerst/histo-projects/CellViT/results/PanNuke/Revision/CellViT/Common-Loss/SAM-H/Best-Setting/2023-09-10T042813_CellViT-SAM-H-Fold-3"
]


In [34]:
detection_results = []
cell_pq_results = []
tissue_metrics = []
global_f1 = []
global_prec = []
global_rec = []


for fold, fold_path in enumerate(fold_paths):
    json_path = Path(fold_path) / "inference_results.json"
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    detection_results.append(pd.DataFrame.from_dict(data["nuclei_metrics_d"]))
    cell_pq_results.append(pd.DataFrame(data["nuclei_metrics_pq"], index=[0]))
    tissue_metrics.append(pd.DataFrame.from_dict(data["tissue_metrics"]))
    global_f1.append(float(data["dataset"]["f1_detection"]))
    global_prec.append(data["dataset"]["precision_detection"])
    global_rec.append(data["dataset"]["recall_detection"])

    
detection_avg = pd.concat([each.stack() for each in detection_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
cell_pq_avg = pd.concat([each.stack() for each in cell_pq_results],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()           
tissue_metrics_avg = pd.concat([each.stack() for each in tissue_metrics],axis=1)\
             .apply(lambda x:x.mean(),axis=1)\
             .unstack()
             
global_f1 = np.mean(global_f1)
global_prec = np.mean(global_prec)
global_rec = np.mean(global_rec)

### Detection-Scores

In [37]:
detection_avg

,Neoplastic,Inflammatory,Connective,Dead,Epithelial
f1_cell,0.705147,0.582973,0.534952,0.362748,0.727920
prec_cell,0.717831,0.592799,0.545667,0.429610,0.723545
rec_cell,0.692981,0.573694,0.524865,0.316230,0.732355


In [38]:
print(f"f1: {global_f1}")
print(f"prec: {global_prec}")
print(f"rec: {global_rec}")

f1: 0.8286793827630442
prec: 0.8431059597223264
rec: 0.8147530828532997


### Cell-Type-PQ-Scores

In [39]:
cell_pq_avg

,Neoplastic,Inflammatory,Connective,Dead,Epithelial
0,0.510192,0.532432,0.546633,0.555504,0.565922


### Tissue-Type-PQ-Scores

In [40]:
tissue_metrics_avg

,adrenal_gland,bile-duct,bladder,breast,cervix,colon,esophagus,headneck,kidney,liver,lung,ovarian,pancreatic,prostate,skin,stomach,testis,thyroid,uterus
Dice,0.844341,0.795063,0.801130,0.836509,0.734218,0.727962,0.841344,0.695361,0.804208,0.870474,0.822380,0.855181,0.852738,0.824733,0.806273,0.873733,0.826648,0.834818,0.848028
Jaccard,0.749195,0.737103,0.751422,0.739229,0.746081,0.687589,0.745632,0.698502,0.756291,0.778295,0.721944,0.752736,0.748346,0.740345,0.710987,0.779146,0.748974,0.753538,0.741031
mPQ,0.546066,0.490967,0.494337,0.524832,0.561727,0.464541,0.553104,0.487259,0.547252,0.582973,0.461155,0.522481,0.537928,0.531873,0.509838,0.532414,0.505622,0.525420,0.494462
bPQ,0.704657,0.638334,0.655327,0.666195,0.593005,0.533350,0.664905,0.555810,0.661817,0.727546,0.631782,0.672118,0.665474,0.663552,0.640628,0.701157,0.673930,0.698241,0.661663


In [42]:
row_mean = tissue_metrics_avg.mean(axis=1)
row_mean

Dice       0.815534
Jaccard    0.741389
mPQ        0.519697
bPQ        0.653131
dtype: float64